In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create an instance of the OpenAIChatCompletionClient
model_client = OpenAIChatCompletionClient(api_key=api_key, model="gpt-4o")

In [3]:
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat

agent_1 = AssistantAgent(
    name='Writer_1',
    model_client=model_client,
    system_message="You are a helpful assistant. Give the output in less than 30 words",
)

agent_2 = AssistantAgent(
    name='Writer_2',
    model_client=model_client,
    system_message="You are a helpful assistant.Give the output in less than 30 words",
)

terminationCondition = MaxMessageTermination(max_messages=3) 

agent_team = RoundRobinGroupChat(participants=[agent_1, agent_2],termination_condition=terminationCondition)

In [4]:
from autogen_agentchat.ui import Console
stream = agent_team.run_stream(task = 'Write a poem about sea in 3 lines')

await Console(stream)

---------- TextMessage (user) ----------
Write a poem about sea in 3 lines
---------- TextMessage (Writer_1) ----------
Endless waves whisper,  
Salt-kissed breezes gently dance,  
Deep blue secrets hide.
---------- TextMessage (Writer_2) ----------
Waves crash on the shore,  
Whispers of stories untold,  
Ocean's vast embrace.


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a poem about sea in 3 lines', type='TextMessage'), TextMessage(source='Writer_1', models_usage=RequestUsage(prompt_tokens=36, completion_tokens=20), metadata={}, content='Endless waves whisper,  \nSalt-kissed breezes gently dance,  \nDeep blue secrets hide.', type='TextMessage'), TextMessage(source='Writer_2', models_usage=RequestUsage(prompt_tokens=64, completion_tokens=21), metadata={}, content="Waves crash on the shore,  \nWhispers of stories untold,  \nOcean's vast embrace.", type='TextMessage')], stop_reason='Maximum number of messages 3 reached, current message count: 3')

In [5]:
team_state = await agent_team.save_state()

In [6]:
await agent_team.reset()

In [7]:
stream = agent_team.run_stream(task = 'What was the last line of the poem you wrote ')

await Console(stream)

---------- TextMessage (user) ----------
What was the last line of the poem you wrote 
---------- TextMessage (Writer_1) ----------
I'm unable to generate or recall previous interactions or content. Could you please provide more details or context about the poem?
---------- TextMessage (Writer_2) ----------
I'm sorry, I can't recall or retrieve past interactions or content. Could you specify which poem you are referring to or provide more information?


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What was the last line of the poem you wrote ', type='TextMessage'), TextMessage(source='Writer_1', models_usage=RequestUsage(prompt_tokens=38, completion_tokens=23), metadata={}, content="I'm unable to generate or recall previous interactions or content. Could you please provide more details or context about the poem?", type='TextMessage'), TextMessage(source='Writer_2', models_usage=RequestUsage(prompt_tokens=69, completion_tokens=27), metadata={}, content="I'm sorry, I can't recall or retrieve past interactions or content. Could you specify which poem you are referring to or provide more information?", type='TextMessage')], stop_reason='Maximum number of messages 3 reached, current message count: 3')

In [ ]:
## Load the state into into new agent

await agent_team.load_state(team_state)
stream = agent_team.run_stream(task = 'What was the last line of the poem you wrote ')

await Console(stream)

---------- TextMessage (user) ----------
What was the last line of the poem you wrote 
---------- TextMessage (Writer_1) ----------
"Deep blue secrets hide."
---------- TextMessage (Writer_2) ----------
The last line I wrote was "Ocean's vast embrace."


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What was the last line of the poem you wrote ', type='TextMessage'), TextMessage(source='Writer_1', models_usage=RequestUsage(prompt_tokens=105, completion_tokens=6), metadata={}, content='"Deep blue secrets hide."', type='TextMessage'), TextMessage(source='Writer_2', models_usage=RequestUsage(prompt_tokens=119, completion_tokens=12), metadata={}, content='The last line I wrote was "Ocean\'s vast embrace."', type='TextMessage')], stop_reason='Maximum number of messages 3 reached, current message count: 3')

In [9]:
## Save state in json and load it back

import json

## Save the state in json
with open('team_state.json', 'w') as f:
    json.dump(team_state, f)

## Load the state from json
with open('team_state.json', 'r') as f:
    team_state = json.load(f)
    

## Reset Team memory for testing
await agent_team.reset()

## Load the state
await agent_team.load_state(team_state)
stream = agent_team.run_stream(task = 'What was the first line of the poem you wrote ')

await Console(stream)

---------- TextMessage (user) ----------
What was the first line of the poem you wrote 
---------- TextMessage (Writer_1) ----------
Endless waves whisper,
---------- TextMessage (Writer_2) ----------
The first line I wrote was: "Waves crash on the shore,"


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What was the first line of the poem you wrote ', type='TextMessage'), TextMessage(source='Writer_1', models_usage=RequestUsage(prompt_tokens=105, completion_tokens=5), metadata={}, content='Endless waves whisper,', type='TextMessage'), TextMessage(source='Writer_2', models_usage=RequestUsage(prompt_tokens=118, completion_tokens=15), metadata={}, content='The first line I wrote was: "Waves crash on the shore,"', type='TextMessage')], stop_reason='Maximum number of messages 3 reached, current message count: 3')